In [2]:
"""
字符编码的策略:
1.one-hot encoding
2.Encode each word with a unique number
3.Word embeddings
"""
# Word embeddings--trainable parameters
# higher dimension -> fine-grained relationships 
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

2023-03-29 18:45:16.264603: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-29 18:45:16.360439: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-29 18:45:16.362454: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 18:45:17.431230: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

['test', 'README', 'imdb.vocab', 'train', 'imdbEr.txt']

In [4]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['pos',
 'urls_neg.txt',
 'urls_unsup.txt',
 'urls_pos.txt',
 'unsupBow.feat',
 'labeledBow.feat',
 'unsup',
 'neg']

In [5]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [6]:
batch_size = 1024
seed = 123
train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


2023-03-29 18:45:53.715973: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [7]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(5):
        print(label_batch[i].numpy(), text_batch.numpy()[i])

2023-03-29 18:45:56.396398: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [20000]
	 [[{{node Placeholder/_4}}]]
2023-03-29 18:45:56.397480: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [20000]
	 [[{{node Placeholder/_4}}]]


0 b"Oh My God! Please, for the love of all that is holy, Do Not Watch This Movie! It it 82 minutes of my life I will never get back. Sure, I could have stopped watching half way through. But I thought it might get better. It Didn't. Anyone who actually enjoyed this movie is one seriously sick and twisted individual. No wonder us Australians/New Zealanders have a terrible reputation when it comes to making movies. Everything about this movie is horrible, from the acting to the editing. I don't even normally write reviews on here, but in this case I'll make an exception. I only wish someone had of warned me before I hired this catastrophe"
1 b'This movie is SOOOO funny!!! The acting is WONDERFUL, the Ramones are sexy, the jokes are subtle, and the plot is just what every high schooler dreams of doing to his/her school. I absolutely loved the soundtrack as well as the carefully placed cynicism. If you like monty python, You will love this film. This movie is a tad bit "grease"esk (without

In [8]:
"""
make sure I/O not become blocking:
1.cache():This will ensure the dataset does not become a 
bottleneck while training your model.If your dataset is too 
large to fit into memory, you can also use this method to create 
a performant on-disk cache, which is more efficient to read than 
many small files.
2.prefetch():overlaps data preprocessing and model execution while training.
"""
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
"""
Embedding layer: map integer indices to dense vectors
"""
# embed 1000 word vocabulary into 5 dimensions
# randomly initialized, during training gradually ajusted to your task
embedding_layer = tf.keras.layers.Embedding(1000, 5)
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()

array([[-0.0083099 ,  0.00552003, -0.03571228,  0.01776811, -0.01485022],
       [ 0.01776758,  0.02911956, -0.04845239, -0.02687578, -0.00223989],
       [ 0.02623161, -0.02557815, -0.04145522, -0.02449559,  0.04076444]],
      dtype=float32)

In [10]:
# batchsize, length, embedding_dimensionality
result = embedding_layer(tf.constant([[0,1,2],[2,3,5]]))
result.shape

TensorShape([2, 3, 5])

In [17]:
"""
Text preprocessing
""" 
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')
vocab_size=10000
sequence_length=100

# Use the text vectorization layer to normalize, split, and map strings to integers
# Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length
# ⭐⭐
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# make a text-only dataset(without labels)
text_ds = train_ds.map(lambda x, y : x)
# call adapt to build the vocabulary
vectorize_layer.adapt(text_ds)
for text_batch in text_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])

2023-03-29 19:23:45.338580: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [20000]
	 [[{{node Placeholder/_0}}]]
2023-03-29 19:23:45.339445: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [20000]
	 [[{{node Placeholder/_4}}]]


b"The original animated Dark Knight returns in this ace adventure movie that rivals Mask of Phantasm in its coolness. There's a lot of style and intelligence in Mystery of the Batwoman, so much more than Batman Forever or Batman and Robin.<br /><br />There's a new crime-fighter on the streets of Gotham. She dresses like a bat but she's not a grown-up Batgirl. And Batman is denying any affiliation with her. Meanwhile Bruce Wayne has to deal with the usual romances and detective work. But the Penguin, Bain and the local Mob makes things little more complicated.<br /><br />I didn't have high hopes for this 'un since being strongly let down but the weak Batman: Sub Zero (Robin isn't featured so much here!)but I was delighted with the imaginative and exciting set pieces, the clever plot and a cheeky sense of humor. This is definitely a movie no fan of Batman should be without. Keep your ears open for a really catchy song called 'Betcha Neva' which is featured prominently through-out.<br /><

2023-03-29 19:23:47.636426: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [20000]
	 [[{{node Placeholder/_4}}]]
2023-03-29 19:23:47.637084: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [20000]
	 [[{{node Placeholder/_0}}]]


In [15]:
"""
Create a classification model:
1.The TextVectorization layer transforms strings into vocabulary indices.
2.The Embedding layer takes the integer-encoded vocabulary and looks up the 
embedding vector for each word-index.
3.The GlobalAveragePooling1D layer returns a fixed-length output vector for 
each example by averaging over the sequence dimension.
"""
embedding_dim = 16

model = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim, name='embedding'),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1)
])

# Compile and train the model
tensorboard_callbacks = tf.keras.callbacks.TensorBoard(log_dir='./TensorBoard/Embedding')
model.compile(optimizer='adam',
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])

model.fit(train_ds,
         validation_data=val_ds,
         epochs=15,
         callbacks=[tensorboard_callbacks])

Epoch 1/15


2023-03-29 19:07:27.037311: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [20000]
	 [[{{node Placeholder/_0}}]]
2023-03-29 19:07:27.038156: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [20000]
	 [[{{node Placeholder/_4}}]]


20/20 [==============================] - ETA: 0s - loss: 0.6920 - accuracy: 0.5028

2023-03-29 19:07:30.536882: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [5000]
	 [[{{node Placeholder/_0}}]]
2023-03-29 19:07:30.537764: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [5000]
	 [[{{node Placeholder/_4}}]]


20/20 [==============================] - 4s 173ms/step - loss: 0.6920 - accuracy: 0.5028 - val_loss: 0.6900 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 3s 143ms/step - loss: 0.6872 - accuracy: 0.5028 - val_loss: 0.6838 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 3s 142ms/step - loss: 0.6791 - accuracy: 0.5028 - val_loss: 0.6739 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 3s 147ms/step - loss: 0.6666 - accuracy: 0.5028 - val_loss: 0.6595 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 3s 151ms/step - loss: 0.6490 - accuracy: 0.5031 - val_loss: 0.6404 - val_accuracy: 0.4914
Epoch 6/15
20/20 [==============================] - 3s 140ms/step - loss: 0.6262 - accuracy: 0.5341 - val_loss: 0.6171 - val_accuracy: 0.5486
Epoch 7/15
20/20 [==============================] - 3s 153ms/step - loss: 0.5988 - accuracy: 0.6029 - val_loss: 0.5908 - val_accuracy: 0.6140
Epoch 8/15
20/20 

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_2 (TextV  (None, 100)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,289
Trainable params: 160,289
Non-trai

In [21]:
"""
retrieve word embedding and save them to disk
weight:(vocab_size, embedding_dimension)
"""
weights= model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

'more'

In [22]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if index == 0:
        continue    # skip 0, it's padding
    vec = weights[index]
    out_v.write('\t'.join([str(x) for x in vec])+"\n")
    out_m.write(word + "\n")
out_v.close()
out_m.close()

In [24]:
model_1 = Sequential()
model_1.add(vectorize_layer)
model_1.predict(['I have lots of money'])

1/1 [==============================] - 0s 147ms/step


array([[ 10,  25, 736,   5, 275,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0]])